# Demo of thesis "Outdoor localization with Wi-Fi"

Author: Darina Dementyeva

In [1]:
import numpy as np
import math
from scipy import stats

## Input data

In [58]:
#room size
n = 100
m = 100

In [57]:
#define params of signal propagation distribution

mu = np.zeros((n, m)) # mean
alpha = 3
d0 = 4
p_d0 = -53

x = np.ones((m,1),int)*range(n)
y = np.transpose(np.ones((n,1),int)*range(m))
d = np.sqrt((x-49)**2 + (y-49)**2)+4
mu = p_d0 - 10 * alpha * np.log10(d / d0)

In [55]:
sigma = np.ones((n, m)) #variance

In [56]:
#start point
x0 = 0
y0 = 0

In [53]:
#start velocity
v0 = 0

In [54]:
#time interval
dt = 1

## Positioning Algorithm

Some model parameters:

In [97]:
#person's motion: acceleration distribution
mu_a = 1
sigma_a = 1

In [60]:
#number of samples
K = 100

Supplementary functions:

In [30]:
def cond_prob(rssi, n, m, mu, sigma):     
    cond_prob = stats.norm(mu, sigma).pdf(rssi)
    return cond_prob

In [36]:
def sample_generation(mu_a, sigma_a, K):
    a = np.random.normal(mu_a, sigma_a, K) #can be changed
    p_a = stats.norm(mu_a, sigma_a).pdf(a)   
    return [a, p_a]

In [98]:
def motion_map(x, y, v, mu_a, sigma_a, K):
    samples  = sample_generation(mu_a, sigma_a, K)
    motion_prob = np.zeros((n, m))
    acceleration = np.zeros((n, m))
    for i in range(0, len(samples[0])):
        a = samples[0][i]
        p_a = samples[1][i]
        x_t = int(round(x + v*dt + a*a*dt/2, 0))
        y_t = int(round(y + v*dt + a*a*dt/2, 0))
        if x_t >= n | y_t >=m: #deal with going beyond the boundaries 
            x_t = x
            y_t = y
            a = 0
        motion_prob[x_t][y_t] = p_a
        acceleration[x_t][y_t] = a
        
    return motion_prob, acceleration

Path calculation: for input we take the array of rssi measurements

In [95]:
def path_estimation(RSSI, x, y, v):
    path_est = []
    path_est.append((0,0))
    #print(str(0) + " " + str(0))
    for rssi in RSSI:
        print("Velocity: " + str(v))
        rssi_prob = cond_prob(rssi, n, m, mu, sigma)
        motion_prob, acceleration = motion_map(x, y, v, mu_a, sigma_a, K)
        prob = np.multiply(rssi_prob, motion_prob)
        best_samples = np.where(prob==prob.max()) #choose not just bust and first but adequate
        x = best_samples[0][0]
        y = best_samples[1][0]
        print(best_samples)
        print(str(x) + " " + str(y))
        a = acceleration[x][y]
        print("Acceleration: " + str(a))
        
        path_est.append((x, y))
        v = v + a*dt
        
    return path_est

## Test generation

In [76]:
def path_generation(length):
    path = [(0,0)]
    rssi = []
    rssi.append(np.random.normal(mu[0][0], sigma[0][0], 1))
    v = 0

    for i in range(1, length):
        a = np.random.normal(1, 1)
        dx = np.random.randint(0, 3) #change to generation with acceleration
        dy = np.random.randint(0, 3)
        x = int(round(path[-1][0] + v*dt + a*a*dt/2, 0))
        y = int(round(path[-1][1] + v*dt + a*a*dt/2, 0))
        v = v + a*dt
        #print(str(dx) + " " + str(dy))
        path.append((x,y))
        rssi.append(np.random.normal(mu[x][y], sigma[x][y], 1))
        
    return rssi, path

In [77]:
N = 5
paths = []

for i in range(0,5):
    paths.append(path_generation(5))

## Model evaluation

In [78]:
def error(path, path_est):
    error = 0
    for i in range(0, len(path)):
        error = error + (path[i][0] - path_est[i][0])**2 + (path[i][1] - path_est[i][1])**2
    error = math.sqrt(error)

    print("Error : " + str(error))

In [79]:
RSSI, path = path_generation(10)
path

[(0, 0),
 (0, 0),
 (0, 0),
 (1, 1),
 (3, 3),
 (6, 6),
 (12, 12),
 (18, 18),
 (27, 27),
 (36, 36)]

In [102]:
path_est = path_estimation(RSSI, x0, y0, v0)

Velocity: 0
(array([2]), array([2]))
2 2
Acceleration: 1.82388149235
Velocity: 1.82388149235
(array([4]), array([4]))
4 4
Acceleration: 0.985725654402
Velocity: 2.80960714675
(array([7]), array([7]))
7 7
Acceleration: -0.237276769939
Velocity: 2.57233037681
(array([10]), array([10]))
10 10
Acceleration: 1.11102510701
Velocity: 3.68335548382
(array([14]), array([14]))
14 14
Acceleration: 0.389048919748
Velocity: 4.07240440357
(array([18]), array([18]))
18 18
Acceleration: -0.527511051439
Velocity: 3.54489335213
(array([22]), array([22]))
22 22
Acceleration: 1.24674811036
Velocity: 4.79164146249
(array([27]), array([27]))
27 27
Acceleration: 1.12489950262
Velocity: 5.91654096511
(array([33]), array([33]))
33 33
Acceleration: 0.551249400647
Velocity: 6.46779036575
(array([39]), array([39]))
39 39
Acceleration: 0.0962496775797


In [103]:
path_est

[(0, 0),
 (2, 2),
 (4, 4),
 (7, 7),
 (10, 10),
 (14, 14),
 (18, 18),
 (22, 22),
 (27, 27),
 (33, 33),
 (39, 39)]

In [104]:
error(path, path_est)

Error : 21.447610589527216
